# Fetching oblique images and metadata from SDFI.
SDFI uses a SpatioTemporal Asset Catalogs: STAC interface. 

#### To use this notebook, you need an environment with the right libraries (specified in the environment.yml file)
To create the environment using conda or mamba type

mamba env create -f environment.yml

The code also assumes that there is a .env file in the same folder as the notebook and that this file takes the form 
token=xxx # where xxx is your dataforsyning  token

## Setting the processing enviorment

In [ ]:
# Imports
from dotenv import dotenv_values
import requests
from urllib.parse import urlparse
import json
import os
from PIL import Image, ImageFile
from fractions import Fraction
from PIL.TiffTags import TAGS
import tifffile
import piexif
import piexif.helper
import csv
from pyproj import Transformer
import xml.etree.ElementTree as ET

Load the token from the .env files

In [ ]:
config  = dotenv_values(".env")
token = config["token"]

In [ ]:
# Common variabuls
base_url = "https://api.dataforsyningen.dk/rest/skraafoto_api/v1.0/"
search_url = base_url + "search"

params = {'token': token}

image_path = "./downloaded_images"
XMP_path= "./downloaded_images"
flight_log_path = "flight_log.JSON"

if not os.path.exists(image_path):
   os.makedirs(image_path)

if not os.path.exists(XMP_path):
   os.makedirs(XMP_path)


# Max number of returnd images can be set up to 1000
limit = 100
# Disable decompression bomb JSON
Image.MAX_IMAGE_PIXELS = None
ImageFile.LOAD_TRUNCATED_IMAGES = True
JPEG_QUALITY = 95  # High-quality JPEG compression

## Setting the search parameters in either WGS or in UTM32

In [ ]:
# Define the bounding box and collection ID in EPSG:23832 coordinates

# Define the bounding box
bbox = [694305.7,  6170219.1 , 694503.5, 6170438.9]
bbox_crs ='http://www.opengis.net/def/crs/EPSG/0/25832'
return_crs = 'http://www.opengis.net/def/crs/EPSG/0/25832'
collection_ids = ["skraafotos2021"]


In [ ]:
# Define the bounding box using wgs84
bbox = [12.087733, 55.639216, 12.089802, 55.640446]
bbox_crs = "http://www.opengis.net/def/crs/OGC/1.3/CRS84"
return_crs = "http://www.opengis.net/def/crs/OGC/1.3/CRS84"
collection_ids = ["skraafotos2021"]


## Fetching the data

### Defining helper procedures

In [ ]:
# Define Helper functions
def download_json_metadata(url, params):
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

def download_image(url, save_path):
    response = requests.get(url, stream=True)
    response.raise_for_status()
    print("Start Download ",url)
    with open(save_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)
    print("download compleat")

def transform_coordinates(x, y, src_crs="EPSG:25832", dest_crs="EPSG:4326"):
    transformer = Transformer.from_crs(src_crs, dest_crs)

    return transformer.transform(x, y)

def to_gps_ifd(val):
    deg = abs(int(val))
    min = abs(int((val % 1) * 60))
    sec = abs(int(((val * 60) % 1) * 60 * 10000))
    return [(deg, 1), (min, 1), (sec, 10000)]

def create_exif_data(metadata):
    # Transform the perspective center coordinates to WGS84
    lat, lon = transform_coordinates(
        metadata["properties"]["pers:perspective_center"][0],
        metadata["properties"]["pers:perspective_center"][1]
    )
    altitude = metadata["properties"]["pers:perspective_center"][2]

    lat_ref = 'N' if lat >= 0 else 'S'
    lon_ref = 'E' if lon >= 0 else 'W'
    
    gps_ifd = {
        piexif.GPSIFD.GPSLatitudeRef: lat_ref,
        piexif.GPSIFD.GPSLatitude: to_gps_ifd(lat),
        piexif.GPSIFD.GPSLongitudeRef: lon_ref,
        piexif.GPSIFD.GPSLongitude: to_gps_ifd(lon),
        piexif.GPSIFD.GPSAltitudeRef: 0 if altitude >= 0 else 1,
        piexif.GPSIFD.GPSAltitude: (int(abs(altitude) * 100), 100),
    }
    
    exif_dict = {
        "0th": {
            piexif.ImageIFD.Make: metadata["properties"]["instruments"][0],
            piexif.ImageIFD.Model: metadata["properties"]["platform"],
            piexif.ImageIFD.DateTime: metadata["properties"]["datetime"],
        },
        "Exif": {
            piexif.ExifIFD.ExposureTime: (1, max(1, int(metadata["properties"]["gsd"]))),
            piexif.ExifIFD.FNumber: (int(metadata["properties"]["estimated_accuracy"]), 1),
            piexif.ExifIFD.FocalLength: (int(metadata["properties"]["pers:interior_orientation"]["focal_length"]), 1),
        },
        "GPS": gps_ifd,
        "1st": {},
        "thumbnail": None,
    }
    
    return exif_dict
    

def save_exif_data(image_path, exif_data):
    exif_bytes = piexif.dump(exif_data)
    jpeg_path = image_path.replace('.tif', '.jpg')
    
    with Image.open(image_path) as img:
        img.convert("RGB").save(jpeg_path, 'JPEG', quality=JPEG_QUALITY, exif=exif_bytes)


        
def create_xmp_file(metadata, save_path):
    lon, lat = transform_coordinates(
        metadata["properties"]["pers:perspective_center"][0],
        metadata["properties"]["pers:perspective_center"][1]
    )
    altitude = metadata["properties"]["pers:perspective_center"][2]

    # Create XML structure
    xmpmeta = ET.Element("x:xmpmeta", {"xmlns:x": "adobe:ns:meta/"})
    rdf = ET.SubElement(xmpmeta, "rdf:RDF", {"xmlns:rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#"})
    description = ET.SubElement(rdf, "rdf:Description", {
        "xcr:Version": "3",
        "xcr:PosePrior": "exact",
        "xcr:Skew": "0",
        "xcr:AspectRatio": "1",
        "xcr:PrincipalPointU": str(metadata["properties"]["pers:interior_orientation"].get("principal_point_offset", [0.0, 0.0])[0]),
        "xcr:PrincipalPointV": str(metadata["properties"]["pers:interior_orientation"].get("principal_point_offset", [0.0, 0.0])[1]),
        "xcr:latitude": f"{abs(lat):.12f}{'N' if lat >= 0 else 'S'}",
        "xcr:longitude": f"{abs(lon):.12f}{'E' if lon >= 0 else 'W'}",
        "xcr:version": "2.2.0.0",
        "xcr:altitude": f"{int(altitude * 10000)}/10000",
        "xmlns:xcr": "http://www.capturingreality.com/ns/xcr/1.1#"
    })
    
    rotation_matrix_str = " ".join(map(str, metadata["properties"]["pers:rotation_matrix"]))
    rotation = ET.SubElement(description, "xcr:Rotation")
    rotation.text = rotation_matrix_str

    # Convert XML structure to string
    xml_data = ET.tostring(xmpmeta, encoding="utf-8").decode("utf-8")

    # Save XML string to file
    with open(save_path, 'w') as file:
        file.write(xml_data)

def save_flight_log(metadata_list, save_path):
    flight_log = {
        "flight_details": [item["properties"] for item in metadata_list]
    }
    with open(save_path, 'w') as file:
        json.dump(flight_log, file, indent=4)


### Main fetch loop

In [ ]:
params = {
    'bbox': ','.join(map(str, bbox)),
    'bbox-crs': bbox_crs,
    'crs':return_crs,
    'collections':collection_ids,
    'limit' : limit,
    'token': token
}

metadata = download_json_metadata(search_url,params)
save_flight_log(metadata["features"], flight_log_path)
if not os.path.exists(image_path):
    os.makedirs(image_path)
if not os.path.exists(XMP_SAVE_PATH):
    os.makedirs(XMP_SAVE_PATH)
for feature in metadata["features"]:
    image_url = feature["properties"]["asset:data"]
    image_id = feature["id"]
    image_path = os.path.join(image_path, f"{image_id}.tif")
    jpeg_path = os.path.join(image_path, f"{image_id}.jpg")
    xmp_path = os.path.join(XMP_SAVE_PATH, f"{image_id}.xmp")
    # Check if the TIFF file already exists
    if not os.path.exists(image_path):
        download_image(image_url, image_path)
    
    # Check if the JPEG file already exists
    if not os.path.exists(jpeg_path):
        # Convert TIFF to JPEG and embed EXIF data
        exif_data = create_exif_data(feature)
        save_exif_data(image_path, exif_data)
        
    if not os.path.exists(xmp_path):
       create_xmp_file(feature, xmp_path)

## List all collections

In [ ]:
# Make the API request to collect all colections
response = requests.get(base_url, params=params)


# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    root_metadata = response.json()
    
    # Extract collections and available services
    collections = [link for link in root_metadata['links'] if link['rel'] == 'child']
    services = [link for link in root_metadata['links'] if link['rel'] != 'child']
    
    # Format and print the collections
    print("LIst of current collections:")
    for collection in collections:
        print(f"Title: {collection['title']}")
        print(f"URL: {collection['href']}")
        print()
    
else:
    print(f"Failed to retrieve data: {response.status_code}")


For a definition on oblique image data see
Verykokou, Styliani, and Charalabos Ioannidis. 2024. "Oblique Aerial Images: Geometric Principles, Relationships and Definitions" Encyclopedia 4, no. 1: 234-255. https://doi.org/10.3390/encyclopedia4010019 

## Image normalisation
To help with the photogeomatry process the following cells normalises the coilurs in the downloaded images

In [ ]:
import cv2
import os
from glob import glob

def normalize_image(image_path):
    # Read the image using OpenCV
    image = cv2.imread(image_path)
    
    # Convert the image from BGR to LAB color space
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    
    # Split the LAB image into L, A and B channels
    l_channel, a_channel, b_channel = cv2.split(lab_image)
    
    # Apply histogram equalization to the L channel (brightness)
    l_channel_eq = cv2.equalizeHist(l_channel)
    
    # Merge the channels back
    lab_image_eq = cv2.merge((l_channel_eq, a_channel, b_channel))
    
    # Convert back to BGR color space
    normalized_image = cv2.cvtColor(lab_image_eq, cv2.COLOR_LAB2BGR)
    
    return normalized_image

def process_images(image_folder):
    # Find all JPG and TIF images in the folder
    image_paths = glob(os.path.join(image_folder, '*.[jJ][pP][gG]')) + \
                  glob(os.path.join(image_folder, '*.[tT][iI][fF]'))
    
    for image_path in image_paths:
        # Normalize the image
        normalized_img = normalize_image(image_path)
        
        # Create the output path by adding "_normalized" to the filename
        base_name = os.path.basename(image_path)
        file_name, ext = os.path.splitext(base_name)
        output_path = os.path.join(image_folder, f"{file_name}_normalized{ext}")
        
        # Save the normalized image
        cv2.imwrite(output_path, normalized_img)
        print(f"Saved normalized image to {output_path}")

# Example usage
process_images(image_path)
